In [26]:
import pandas as pd
from Bio import SeqIO
import os
import re

guide_gene_names = dict()

with open('1k_ascomycota_library.txt', 'r') as f:
    for l in f.readlines():
        guide_gene_names[l.strip()] = list()

files = os.listdir('out')
ortholog_re = r'\[orthologous_to_gene=(.*?)\]'

for f in files:
    guide_target_names = dict()

    with open(f'out/{f}', 'r') as inf:
        lines = inf.readlines()

        for l in lines:
            if l.startswith(','):
                continue
            
            splited = l.split(',')

            guide_target_names.setdefault(splited[1][:20], set()).add(splited[3])

    for k, vlist in guide_target_names.items():
        try:
            records = SeqIO.parse(f'target_sequence/{f}', 'fasta')
        except FileNotFoundError:
            continue
        
        for r in records:
            name = r.description.split(' ')

            if name[0] in vlist or name[1] in vlist:
                ortholog_re_match = re.search(ortholog_re, r.description)

                if ortholog_re_match:
                    ortho_gene = ortholog_re_match.group(1)

                    guide_gene_names[k].append(ortho_gene)

guide_gene_names

{'AGAAGAGGAGGAGGAGGAGG': ['RHB1',
  'NTA1',
  'COQ8',
  'FUN30',
  'ECM10',
  'PPE1',
  'RSC1',
  'PDR3',
  'YOS9',
  'LSB5',
  'CSH1',
  'PET127',
  'NIP1',
  'BOL3',
  'RSC1',
  'DAL2',
  'DPB4',
  'TSR3',
  'GIP4',
  'STU2',
  'PXR1',
  'VPS62',
  'ASF1',
  'SIM1',
  'SPT5',
  'ABP140',
  'EBP2',
  'MGM1',
  'STH1',
  'SSD1',
  'GYP7',
  'RQC2',
  'SDH5',
  'CYC2',
  'SSF2',
  'SWH1',
  'YOS9',
  'CKB1',
  'RNA1',
  'PAH1',
  'CDC34',
  'RAD26',
  'SPT5',
  'SEC9',
  'DIN7',
  'RSA4',
  'ASF1',
  'ZDS2',
  'AKR2',
  'RSA3',
  'MPS1',
  'YHI9',
  'CAB3',
  'TRX3',
  'KRI1',
  'FUS3',
  'ESF1',
  'QCR6',
  'ESP1',
  'QCR6',
  'RPC19',
  'VID27',
  'SFH5',
  'SDT1',
  'RSC1',
  'UTP4',
  'BYE1',
  'FRE4',
  'KCS1',
  'CTI6',
  'IPA1',
  'QCR6',
  'LSG1',
  'ADP1',
  'PTH2',
  'WWM1',
  'ART5',
  'DOA1',
  'DUN1',
  'USO1',
  'RAD54',
  'BUD6',
  'SSD1',
  'SRP40',
  'TCB2',
  'FUN30',
  'RNA1',
  'SAS3',
  'NOT3',
  'QCR6',
  'LSB5',
  'CSH1',
  'TLG1',
  'ECM13',
  'NAS6',
  'VID27',


In [20]:
guide_gene_names.keys()

dict_keys(['AGAAGAGGAGGAGGAGGAGG', 'AGAGGAAGAGGAAGAGGAAG', 'ATCCAATGGAGATTCCAAAC', 'GAGGCTGGTATCTCCAAGGA', 'GCTGGTGACACTCACTTGGG', 'GCTTTCTTGCATTGGTACAC', 'TCTTCATCGGCTTCGGCGAA', 'TGCTGTTGCTGTTGCTGCTG'])

In [68]:
from collections import Counter
import matplotlib.pyplot as plt

def count_and_sort(guide_gene_names, gene_sequence):
    counts = Counter(guide_gene_names[gene_sequence])
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    return zip(*sorted_counts)

guide_gene_sequences = [
    'AGAAGAGGAGGAGGAGGAGG',
    'AGAGGAAGAGGAAGAGGAAG',
    'ATCCAATGGAGATTCCAAAC',
    'GAGGCTGGTATCTCCAAGGA',
    'GCTGGTGACACTCACTTGGG',
    'GCTTTCTTGCATTGGTACAC',
    'TCTTCATCGGCTTCGGCGAA',
    'TGCTGTTGCTGTTGCTGCTG'
]

labels_sizes = {}
for seq in guide_gene_sequences:
    labels_sizes[seq] = list(count_and_sort(guide_gene_names, seq))

def create_pie_chart(ax, labels, sizes, title):
    if len(labels) > 5:
        upto = 5
        small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
        small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']
    else:
        small_sizes = list(sizes)
        small_labels = list(labels)

    colors = ['#FFC6FF', '#9BF6FF', '#CAFFBF', '#D97518', '#FFADAD', '#FDFFB6']
    wedges, texts, autotexts = ax.pie(small_sizes, colors=colors, autopct='%1.1f%%', startangle=140, textprops=dict(color="b"))

    legend_labels = [f"{label} ({size})" for label, size in zip(small_labels, small_sizes)]
    ax.legend(wedges, legend_labels, title="Items", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

    plt.setp(autotexts, size=10, weight="bold")
    ax.set_title(title)

plt.rcParams['figure.dpi'] = 600

# Prepare figure and subplots
fig, axs = plt.subplots(4, 2, figsize=(16, 16))

# Titles for each subplot
titles = [
    'AGAAGAGGAGGAGGAGGAGG',
    'AGAGGAAGAGGAAGAGGAAG',
    'ATCCAATGGAGATTCCAAAC',
    'GAGGCTGGTATCTCCAAGGA',
    'GCTGGTGACACTCACTTGGG',
    'GCTTTCTTGCATTGGTACAC',
    'TCTTCATCGGCTTCGGCGAA',
    'TGCTGTTGCTGTTGCTGCTG'
]

for ax, seq in zip(axs.flat, guide_gene_sequences):
    labels, sizes = labels_sizes[seq]
    create_pie_chart(ax, labels, sizes, seq)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [46]:
labels_sizes.keys()

dict_keys(['AGAAGAGGAGGAGGAGGAGG', 'AGAGGAAGAGGAAGAGGAAG', 'ATCCAATGGAGATTCCAAAC', 'GAGGCTGGTATCTCCAAGGA', 'GCTGGTGACACTCACTTGGG', 'GCTTTCTTGCATTGGTACAC', 'TCTTCATCGGCTTCGGCGAA', 'TGCTGTTGCTGTTGCTGCTG'])

In [48]:
labels, sizes = labels_sizes['AGAAGAGGAGGAGGAGGAGG']

upto = 5
small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']

In [50]:
small_labels

['QCR6', 'RNA1', 'VID27', 'EBP2', 'EAF7', '335 Other Genes']

In [55]:
small_sizes

[23, 19, 12, 8, 8, 517]

In [56]:
labels, sizes = labels_sizes['AGAGGAAGAGGAAGAGGAAG']

upto = 5
small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']

In [57]:
small_labels

['QCR6', 'PPN1', 'UTP25', 'TPA1', 'APL6', '291 Other Genes']

In [58]:
small_sizes

[10, 9, 7, 7, 7, 454]

In [59]:
labels, sizes = labels_sizes['ATCCAATGGAGATTCCAAAC']

upto = 5
small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']

In [60]:
small_labels

['YOR1', 'PXA2', 'PDR15', 'VMR1', 'STE6', '5 Other Genes']

In [61]:
small_sizes

[24, 12, 5, 4, 3, 7]

In [62]:
labels, sizes = labels_sizes['GAGGCTGGTATCTCCAAGGA']

upto = 5
small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']

In [63]:
small_labels

['TEF2', 'SUP35', 'FUN12', 'MRL1', 'PFK26', '3 Other Genes']

In [64]:
small_sizes

[709, 3, 1, 1, 1, 3]

In [65]:
labels, sizes = labels_sizes['GCTGGTGACACTCACTTGGG']

upto = 5
small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']

In [66]:
small_labels

['SSA1', 'SSB1', 'ECM10', 'KAR2', 'SSA3', '6 Other Genes']

In [67]:
small_sizes

[315, 22, 9, 7, 4, 8]

In [69]:

labels, sizes = labels_sizes['GCTTTCTTGCATTGGTACAC']

upto = 5
small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']

In [70]:
small_labels

['RIB5', 'TUB3', 'DML1', 'MRPL6', 'TUB4', '2 Other Genes']

In [74]:
small_sizes

43.29710144927536

In [75]:
labels, sizes = labels_sizes['TCTTCATCGGCTTCGGCGAA']

upto = 5
small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']

In [76]:
small_sizes

[20, 6, 0]

In [77]:
small_labels

['SUI1', 'PRP24', '-3 Other Genes']

In [83]:
labels, sizes = labels_sizes['TGCTGTTGCTGTTGCTGCTG']

upto = 5
small_sizes = list(sizes[:upto]) + [sum(sizes[upto:])]
small_labels = list(labels[:upto]) + [f'{len(labels) - upto} Other Genes']

In [85]:
small_labels

['ENT1', 'GLN3', 'MIG3', 'SNT1', 'SMP1', '92 Other Genes']

In [86]:
small_sizes

[5, 5, 4, 4, 4, 117]